In [1]:
import sys
import imp
sys.path.insert(0, '../src')

In [2]:
import networkit as nk
from denoise import networkio
imp.reload(networkio)

G, node_list, node_map = networkio.parse_dream_network_file("../data/1_ppi_string_v2.txt")

print(G.weight(node_map["PRKAG2"], node_map["ZNF559-ZNF177"])) # Should output 0.064
print(G.weight(node_map["ARL5C"], node_map["YKT6"]))           # Should output 0.15

_, proteins_to_go = main.parse_go_label_file("../data/human.golabels")

print(proteins_to_go["PRKAG2"]) # Works as expected
print(proteins_to_go["YKT6"])   # Works as expected

FileNotFoundError: [Errno 2] No such file or directory: '../data/1_ppi_string_v2.txt'

In [3]:
from denoise import predict
imp.reload(predict)

G = nk.Graph(n=6, weighted=True)
M = 1_000_000

# Node names and labels
A, B, C, D, E, F = range(0, 6)
labels = {
    A: ["P3"],
    B: ["P1", "P2"],
    E: ["P3", "P4"],
    F: ["P4", "P2"]
}
labels_f = lambda n: (labels[n] if n in labels else []) 

G.addEdge(A, B, M)
G.addEdge(B, C, 4)
G.addEdge(A, C, 1)
G.addEdge(D, A, 3)
G.addEdge(D, C, M)
G.addEdge(E, D, 1)

predict.wmv(G, labels_f)

{0: 'P3', 1: 'P1', 2: 'P1', 3: 'P3', 4: 'P3', 5: 'P4'}

In [60]:
G, node_list, node_map = networkio.parse_dream_network_file("../data/1_ppi_string_v2.txt")
_, proteins_to_go = main.parse_go_label_file("../data/human.golabels")

In [4]:
from denoise import scoring
from denoise import predict
imp.reload(predict)
imp.reload(scoring)

count, total = 0, 0
for n in G.iterNodes():
    if node_list[n] not in proteins_to_go:
        count += 1
    total += 1
print(f"Unlabelled over labelled: {count}/{total}")

labels = {n: proteins_to_go[node_list[n]] for n in G.iterNodes() 
          if node_list[n] in proteins_to_go}

def predictor(training_labels):
    tlabels_f = lambda n: (training_labels[n] if n in training_labels else [])
    return predict.wmv(G, tlabels_f)

scoring.kfoldcv(50, labels, predictor, randomized=True)

NameError: name 'node_list' is not defined